In [1]:
from genet import predict_dev as prd

list_target30 = [
                'TCACCTTCGTTTTTTTCCTTCTGCAGGAGG',
                'CCTTCGTTTTTTTCCTTCTGCAGGAGGACA',
                'CTTTCAAGAACTCTTCCACCTCCATGGTGT',
                ]


df = prd.spcas9_score_tf2(list_target30)
df


The model DeepSpCas9 is not installed. Download checkpoint files.



Downloading: 0KB [00:00, ?KB/s]


File downloaded successfully: c:\Users\home\Documents\GitHub\genet\genet\models\DeepSpCas9/__init__.py


Downloading: 681KB [00:00, 1495.85KB/s]                         


File downloaded successfully: c:\Users\home\Documents\GitHub\genet\genet\models\DeepSpCas9/PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60.data-00000-of-00001


Downloading: 1KB [00:00, ?KB/s]


File downloaded successfully: c:\Users\home\Documents\GitHub\genet\genet\models\DeepSpCas9/PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60.index


Downloading: 29KB [00:00, 240.87KB/s]                        


File downloaded successfully: c:\Users\home\Documents\GitHub\genet\genet\models\DeepSpCas9/PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60.meta


OSError: SavedModel file does not exist at: c:\Users\home\Documents\GitHub\genet\genet\models\DeepSpCas9\{saved_model.pbtxt|saved_model.pb}

In [1]:
from genet.predict_dev import spcas9_save

spcas9_save('./')



Type of model <class 'genet.predict_dev.functional_dev.Deep_xCas9'>


ValueError: Expected `model` argument to be a `Model` instance. Received: model=<class 'genet.predict_dev.functional_dev.Deep_xCas9'>

In [2]:
from genet.models import LoadModel

model = LoadModel('SpCas9')

The model DeepSpCas9 is not installed. Download checkpoint files.


Downloading: 0KB [00:00, ?KB/s]


File downloaded successfully: e:\github_project\genet\genet\models\DeepSpCas9/__init__.py


Downloading: 681KB [00:00, 837.64KB/s]                         


File downloaded successfully: e:\github_project\genet\genet\models\DeepSpCas9/PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60.data-00000-of-00001


Downloading: 1KB [00:00, 496.78KB/s]


File downloaded successfully: e:\github_project\genet\genet\models\DeepSpCas9/PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60.index


Downloading: 29KB [00:00, 221.38KB/s]                      

File downloaded successfully: e:\github_project\genet\genet\models\DeepSpCas9/PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60.meta


In [ ]:
seq_wt   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'
seq_ed   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'
alt_type = 'sub1'

df_pe = prd.pe_score(seq_wt, seq_ed, alt_type)
df_pe.head()

In [ ]:
df_test = df_pe['Edited74_On'].str.replace('x', '')
df_test

In [ ]:
selected = ['WT74_On', 'PBSlen', 'RTlen', 'RT-PBSlen', 'Edit_pos', 'Edit_len', 'RHA_len', 'PE2max_score']
df_pe_summary = df_pe[selected]
df_pe_summary.insert(1, 'RT-PBSseq', list_rtpbs)

In [ ]:
# genet ToDo

# 1. output dataframe 간단하게 표현하기
# 2. GetGene 불러와서 Base editing용 gRNA 디자인해주기

In [1]:
import logging

class LogStreamHandler(logging.StreamHandler):
    def __init__(self, stream=None):
        super().__init__(stream)
        self.setFormatter(logging.Formatter(
            '%(levelname)-5s @ %(asctime)s:\n\t %(message)s \n',
            datefmt='%a, %d %b %Y %H:%M:%S',
        ))
        self.setLevel(logging.INFO)

class StatusFormatter(logging.Formatter):
    def format(self, record):
        record.percent_complete = ''
        if record.args and 'percent_complete' in record.args:
            record.percent_complete = '{0:.2f}% '.format(record.args['percent_complete'])
            self.last_percent_complete = record.percent_complete
        elif hasattr(self, 'last_percent_complete'): # if we don't have a percent complete, use the last one
            record.percent_complete = self.last_percent_complete
        return super().format(record)

class StatusHandler(logging.FileHandler):
    def __init__(self, filename):
        super().__init__(filename, 'w')
        self.setFormatter(StatusFormatter('%(percent_complete)s%(message)s'))

    def emit(self, record):
        """Overwrite the existing file and write the new log."""
        if self.stream is None:  # log file is empty
            self.stream = self._open()
        else:  # log file is not empty, overwrite
            self.stream.seek(0)
        logging.StreamHandler.emit(self, record)
        self.stream.truncate()


In [4]:
import logging, os, sys
import genet

class DeepPrime:
    '''
    DeepPrime: pegRNA activity prediction models\n
    Input  = 121 nt DNA sequence without edit\n
    Output = 121 nt DNA sequence with edit\n
    
    ### Available Edit types\n
    sub1, sub2, sub3, ins1, ins2, ins3, del1, del2, del3\n
    
    ### Available PE systems\n
    PE2, PE2max, PE4max, NRCH_PE2, NRCH_PE2max, NRCH_PE4max\n
    
    ### Available Cell types\n
    HEK293T, HCT116, MDA-MB-231, HeLa, DLD1, A549, NIH3T3
    
    '''
    def __init__(self, sID:str, Ref_seq: str, ED_seq: str, edit_type: str, edit_len: int,
                pam:str = 'NGG', pbs_min:int = 7, pbs_max:int = 15,
                rtt_min:int = 0, rtt_max:int = 40, silence:bool = False,
                out_dir:str=os.getcwd(),
                ):
        
        # input parameters
        self.nAltIndex = 60
        self.sID, self.Ref_seq, self.ED_seq = sID, Ref_seq, ED_seq
        self.edit_type, self.edit_len, self.pam = edit_type, edit_len, pam
        self.pbs_min, self.pbs_max = pbs_min, pbs_max
        self.pbs_range = [pbs_min, pbs_max]
        self.rtt_min, self.rtt_max   = rtt_min, rtt_max
        self.silence = silence
        
        # output directory
        self.OUT_PATH = '%s/%s/'  % (out_dir, self.sID)
        self.TEMP_DIR = '%s/temp' % self.OUT_PATH
        
        # initializing
        self.set_logging()
        self.check_input()

        ## FeatureExtraction Class
        cFeat = FeatureExtraction()

        cFeat.input_id = sID
        cFeat.get_input(Ref_seq, ED_seq, edit_type, edit_len)

        cFeat.get_sAltNotation(nAltIndex)
        cFeat.get_all_RT_PBS(nAltIndex, nMinPBS=pbs_range[0]-1, nMaxPBS=pbs_range[1], nMaxRT=rtt_max, pe_system=pe_system)
        cFeat.make_rt_pbs_combinations()
        cFeat.determine_seqs()
        cFeat.determine_secondary_structure()

        self.features = cFeat.make_output_df()
        
        del cFeat


        self.logger.info('Created an instance of DeepPrime')

    # def __init__: END


    def submit(self, pe_system:str, cell_type:str = 'HEK293T'):
        print('start pe_scre', self.Ref_seq, self.ED_seq, )

        return None

    # def submit: END


    def set_logging(self):

        self.logger = logging.getLogger(self.OUT_PATH)
        self.logger.setLevel(logging.DEBUG)

        self.formatter = logging.Formatter(
            '%(levelname)-5s @ %(asctime)s:\n\t %(message)s \n',
            datefmt='%a, %d %b %Y %H:%M:%S',
            )
        
        self.error = self.logger.error
        self.warn  = self.logger.warn
        self.debug = self.logger.debug
        self.info  = self.logger.info

        try:
            os.makedirs(self.OUT_PATH, exist_ok=True)
            os.makedirs(self.TEMP_DIR, exist_ok=True)
            self.info('Creating Folder %s' % self.OUT_PATH)
        except:
            self.error('Creating Folder failed')
            sys.exit(1)
            
        self.file_handler = logging.FileHandler('%s/log_%s.log' % (self.OUT_PATH, self.sID))
        self.file_handler.setLevel(logging.DEBUG)
        self.file_handler.setFormatter(self.formatter)
        self.logger.addHandler(self.file_handler)
        
        if self.silence != True:
            self.console_handler = logging.StreamHandler()
            self.console_handler.setLevel(logging.DEBUG)
            self.console_handler.setFormatter(self.formatter)
            self.logger.addHandler(self.console_handler)

        self.info('DeepPrime: pegRNA activity prediction models\n\t version: %s' % genet.__version__)


        return None

    # def set_logging: END


    def check_input(self):
        
        if self.pbs_min < 1:
            self.error('sID:%s\nPlease set PBS max length at least 1nt' % self.sID)
            raise ValueError('Please check your input: pbs_min')
        
        if self.pbs_max > 17:
            self.error('sID:%s\nPlease set PBS max length upto 17nt' % self.sID)
            raise ValueError('Please check your input: pbs_max')
        
        if self.rtt_max > 40:
            self.error('sID:%s\nPlease set RTT max length upto 40nt' % self.sID)
            raise ValueError('Please check your input: rtt_max')

        if self.edit_type not in ['sub', 'ins', 'del']:
            self.error('sID:%s\n\t Please select proper edit type.\n\t Available edit tyle: sub, ins, del' % self.sID)
            raise ValueError('Please check your input: edit_type')

        if self.edit_len > 3:
            self.error('sID:%s\n\t Please set edit length upto 3nt. Available edit length range: 1~3nt' % self.sID)
            raise ValueError('Please check your input: edit_len')
        
        if self.edit_len < 1:
            self.error('sID:%s\n\t Please set edit length at least 1nt. Available edit length range: 1~3nt' % self.sID)
            raise ValueError('Please check your input: edit_len')

        self.info('Input information\n\t ID: %s\n\t Refseq: %s\n\t EDseq :%s' % (self.sID, self.Ref_seq, self.ED_seq))

        return None
    
    # def check_input: END


    def do_something(self):
        self.logger.info('Something happened.')

        return None

    # def do_something: END
    



In [5]:
ins_a = DeepPrime(sID='Sample_1',
                  Ref_seq='ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT', 
                  ED_seq='ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT',
                  edit_type='sub',
                  edit_len=1)

INFO  @ Tue, 02 May 2023 19:22:44:
	 DeepPrime: pegRNA activity prediction models
	 version: 0.5.2 

INFO  @ Tue, 02 May 2023 19:22:44:
	 Input information
	 ID: Sample_1
	 Refseq: ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT
	 EDseq :ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT 

INFO  @ Tue, 02 May 2023 19:22:44:
	 Created an instance of DeepPrime 



In [6]:
ins_a.do_something()

INFO  @ Tue, 02 May 2023 19:22:45:
	 Something happened. 



In [7]:
ins_a.do_something()

INFO  @ Tue, 02 May 2023 19:22:46:
	 Something happened. 



In [8]:
ins_b = DeepPrime(sID='Sample_2',
                  Ref_seq='ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT', 
                  ED_seq='ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT',
                  edit_type='sub',
                  edit_len=1,
                  silence=True)

In [9]:
ins_b.do_something()

In [10]:
ins_c = DeepPrime(sID='Sample_3',
                  Ref_seq='ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT', 
                  ED_seq='ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT',
                  edit_type='sub',
                  edit_len=4,
                  )

INFO  @ Tue, 02 May 2023 19:22:54:
	 DeepPrime: pegRNA activity prediction models
	 version: 0.5.2 

ERROR @ Tue, 02 May 2023 19:22:54:
	 sID:Sample_3
	 Please set edit length upto 3nt. Available edit length range: 1~3nt 

INFO  @ Tue, 02 May 2023 19:22:54:
	 Input information
	 ID: Sample_3
	 Refseq: ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT
	 EDseq :ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT 

INFO  @ Tue, 02 May 2023 19:22:54:
	 Created an instance of DeepPrime 



In [10]:
ins_c.do_something()

INFO  @ Tue, 02 May 2023 19:17:52:
	 Something happened. 



In [8]:
import sys, logging


logger = logging.getLogger('example')


raise ValueError('Error!@@!@!@')

logger.error('Error')



print('Error occured')
sys.exit(1)

ValueError: Error!@@!@!@

In [1]:
from genet.predict import CasVariant

cas_ng = CasVariant('SpCas9-NG')

c:\Users\gsyu\miniconda3\envs\genet\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
list_target30 = [
                'TCACCTTCGTTTTTTTCCTTCTGCAGGAGG',
                'CCTTCGTTTTTTTCCTTCTGCAGGAGGACA',
                'CTTTCAAGAACTCTTCCACCTCCATGGTGT',
                ]

df_out = cas_ng.predict(list_target30)
df_out


,Sequence,SpCas9-NG
0,TCACCTTCGTTTTTTTCCTTCTGCAGGAGG,0.618299
1,CCTTCGTTTTTTTCCTTCTGCAGGAGGACA,1.134845
2,CTTTCAAGAACTCTTCCACCTCCATGGTGT,36.743580
